Домашнее задание к лекции "Основы веб-скрапинга и работы с API"
Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
KEYWORDS = ['python', 'парсинг']
URL = 'https://habr.com/ru/search/?'


In [35]:
post_times = []
post_titles = []
post_links = []
post_body = []
for keyword in KEYWORDS:
    params = {
        'q':keyword
    }
    req = requests.get(URL, params)
    soup = BeautifulSoup(req.text, 'html.parser')
    posts = soup.find_all('article', class_='post')
    for post in posts:
        post_times.append(post.find('span', class_='post__time').text)
        post_titles.append(post.find('h2', class_='post__title').text)
        post_links.append(post.find('a', class_='btn').get('href'))
        req2 = requests.get(post.find('a', class_='btn').get('href'))
        content = BeautifulSoup(req2.text, 'html.parser')
        post_body.append(content.find('div', class_ = 'post__text').text.strip())
        
post_titles = list(map(lambda x: x.strip(), post_titles))
articles = pd.DataFrame({'время':post_times, 'заголовок':post_titles, 'ссылка':post_links, 'текст': post_body})
articles.head()

,время,заголовок,ссылка,текст
0,21 апреля 2020 в 18:35,"Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/t/498364/#habracut,"20 апреля 2020 года, спустя почти десять лет п..."
1,3 марта 2020 в 13:22,... начале этого года Python сместил Java и ...,https://habr.com/ru/company/itsumma/news/t/490...,"Согласно отчету RedMonk за январь 2020 года, P..."
2,3 июня 2020 в 09:00,Вышла версия 0.0.2 snakeware — дистрибутива Li...,https://habr.com/ru/news/t/505096/#habracut,31 мая 2020 года разработчик Джош Мур (Josh Mo...
3,вчера в 10:10,Создатель Python Гвидо ван Россум выходит на р...,https://habr.com/ru/news/t/527858/#habracut,Создатель языка программирования Python Гвидо ...
4,31 октября 2019 в 10:02,Создатель Python Гвидо ван Россум ушел из Drop...,https://habr.com/ru/news/t/473926/#habracut,Создатель языка программирования Python Гвидо ...


Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [9]:
emails = []
sites = []
publishDates = []
descriptions = []
emailsToCheck = ['xxx@xxx.com', 'yyy@yyy.com']
for emailToCheck in emailsToCheck:
    data = {"emailAddresses":[emailToCheck]}
    url1 = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
    response = requests.post(url1, json = data, headers = {'Vaar-Header-App-Product': 'hackcheck-web-avast','Vaar-Version': '0'})
    for site in response.json()['data'].keys():
        time.sleep(0.2)
        breachId = str(list(response.json()['data'][site].values())[0][0]['breachId'])
        publishDate = response.json()['breaches'][breachId]['publishDate']
        description = response.json()['breaches'][breachId]['description']
        email = list(response.json()['data'][site].keys())[0]
        emails.append(email)
        sites.append(site)
        publishDates.append(publishDate)
        descriptions.append(description)
breachesReport = pd.DataFrame ({'Почта' : emails, 'Кто слил' : sites, 'Когда' : publishDates, 'Как так вышло' : descriptions})
breachesReport.tail()

,Почта,Сайт,Дата,Описание
120,yyy@yyy.com,tumblr.com,2016-10-25T00:00:00Z,"In May 2016, microblogging platform Tumblr rev..."
121,yyy@yyy.com,shein.com,2019-05-16T00:00:00Z,"In June 2018, fast-fashion retailer Shein's da..."
122,yyy@yyy.com,zynga.com,2019-10-17T00:00:00Z,"In September 2019, the game developer Zynga wa..."
123,yyy@yyy.com,toondo.com,2019-11-14T00:00:00Z,"In July 2019, the online comic creation site T..."
124,yyy@yyy.com,sharperimage.com,2019-02-21T00:00:00Z,"This is a list of email addresses only, and as..."
